In [1]:
import pandas as pd
import numpy as np
from ta import add_all_ta_features

%run C:\Projects\Stock_price_prediction\Experiments\data_processing_functions.ipynb
%run C:\Projects\Stock_price_prediction\Experiments\feature_engineering.ipynb

# 2019-2024 data

In [2]:
# For importing pre-processed data and further process here
data_path = 'C:/Projects/Stock_price_prediction/Experiments/WRDS_raw_20190101-20240531.csv'
TBill_path = 'C:/Projects/Stock_price_prediction/3monthTBill_2006-03-01_2024-07-12.csv'
sic_code_path = 'C:/Projects/Stock_price_prediction/Experiments/tickers&sics_20200101-20240531.csv'

In [3]:
data = pd.read_csv(data_path)

data['datadate'] = pd.to_datetime(data['datadate'], format='%Y-%m-%d')
data = data.sort_values(by=['tic', 'datadate']).reset_index(drop=True)

total_dates = set(data['datadate'].unique())
apple_dates = set(data[data['tic']=='AAPL']['datadate'].unique())
print(len(apple_dates))
print(len(total_dates - apple_dates))

# Uses the numbers of days for Apple as the standard and drop any other stock with fewer dates
num_of_ts = len(data[data['tic']=='AAPL']['datadate'].unique())
factors = ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod']
print(f'There are {num_of_ts} timestamps')

tickers = list(data.tic.unique())
print(f'There are {len(tickers)} tickers')

# Removes stocks with missing data
data, tickers = remove_tic(data, tickers, num_of_ts, factors)

# Check that each ticker has exactly num_of_ts timestamps
assert data.shape[0] == num_of_ts * len(tickers)

data, tickers = remove_low_dollar_vol(data, tickers, dol_vol_thres = 10000000)

data = data.reset_index(drop=True)
data, tickers = adjust_split(data, tickers, factors)

data = compute_ret(data, TBill_path)
data

1363
46
There are 1363 timestamps
There are 25633 tickers
Removed ['3RELLB', 'AAM.PB', 'AATRL', 'ABAKF', 'ABCM', 'ABR.PD', 'ABR.PF', 'ABST', 'ACC', 'ACCFF', 'ACFL', 'ACHHY', 'ACIA', 'ACLTF', 'ACRVF', 'ADFJF', 'ADKT', 'ADLRF', 'ADWPF', 'AETUF', 'AFAP', 'AFCG', 'AFHBQ', 'AFLYY', 'AFSI.PB', 'AFSI.PC', 'AFSI.PD', 'AFSI.PE', 'AFSI.PF', 'AFSIA', 'AFTPY', 'AGAAF', 'AGASF', 'AGCBF', 'AGEEF', 'AGGZF', 'AGM.PB', 'AGMJF', 'AGQPF', 'AGTI', 'AHOTF', 'AHTR', 'AIIQ', 'AIMC', 'AJXGF', 'AKZOY', 'ALLXD', 'ALLY.PA', 'ALO', 'ALPIB', 'ALPN', 'ALQA', 'ALSMY', 'ALTG.A', 'ALTGF', 'AMIVF', 'AMKBY', 'AMSSY', 'ANDHF', 'AOCIF', 'APGN', 'APHLF', 'APMCF', 'APO.PB', 'APPTF', 'APRN', 'APRRF', 'APRWF', 'APYRF', 'AQNU', 'AQUA', 'ARCE', 'ARESF', 'ARGGY', 'ARGO', 'ARGO.PA', 'ARI.PC', 'ARNA', 'ARREF', 'ARSMF.', 'ASCCY', 'ASFI', 'ASGTF', 'AT', 'ATADF', 'ATBYF', 'ATCD', 'ATCO', 'ATCX', 'ATEYY', 'ATGAF', 'ATGPF', 'ATLMF', 'ATPWF', 'ATRS', 'AUOTY', 'AUTO', 'AUY', 'AVDR.1', 'AVDWF', 'AVGDF', 'AVLR', 'AVPFF', 'AVVYY', 'AWRRF', 

Removed ['CHKVQ', 'CNIGP', 'FNMFO', 'KPFSY', 'NLY.PC', 'PLL.2', 'PRLPQ', 'SBNCM'] for missing prchd
Removed ['BORNY', 'CNTFY', 'LASLY', 'MDLV', 'REDFY', 'WCAGY'] for missing prcld
Removed ['AAALF', 'AAALY', 'AABVF', 'AAC.U', 'AAC.WS', 'AACAY', 'AACG', 'AACH', 'AACI', 'AACIU', 'AACIW', 'AACQU', 'AACS', 'AACT', 'AACT.U', 'AACT.WS', 'AACTF', 'AADR', 'AAEEF', 'AAFRF', 'AAGC', 'AAGFF', 'AAGH', 'AAGIY', 'AAGR', 'AAGRW', 'AAIC.PB', 'AAIC.PC', 'AAIIQ', 'AAMC', 'AAME', 'AAMMF', 'AAMTF', 'AANNF', 'AAPI', 'AAPJ', 'AAPT', 'AAQC', 'AAQCU', 'AAQCW', 'AARTY', 'AASP', 'AATC', 'AATV', 'AAUCF', 'AAUGF', 'AAUKF', 'AAVMY', 'AAVVF', 'AAWW', 'ABAHF', 'ABBB', 'ABBNY', 'ABBRF', 'ABBY', 'ABCAF', 'ABCFF', 'ABCP', 'ABCZY', 'ABDR', 'ABEOW', 'ABEPF', 'ABETF', 'ABGI', 'ABHD', 'ABILF', 'ABIT', 'ABL', 'ABLE', 'ABLLL', 'ABLLW', 'ABLT', 'ABLV', 'ABLVW', 'ABLZF', 'ABMBF', 'ABMC', 'ABMT', 'ABNAF', 'ABNK', 'ABQQ', 'ABR.PA', 'ABR.PB', 'ABR.PC', 'ABR.PE', 'ABRMF', 'ABSSF', 'ABTI', 'ABTI.1', 'ABTS', 'ABVC.1', 'ABVN', 'ABWN',

There are 15388 unique tickers to remove due to missing values
Removed ['3UCPC', 'AAA', 'AABA', 'AAC', 'AACQ', 'AACQW', 'AADI', 'AAIC', 'AAINF', 'AAIR', 'AAM.PA', 'AAN', 'AAPB', 'AAPD', 'AAPH', 'AAPR', 'AAPU', 'AAPX', 'AAPY', 'AAST', 'AAWH', 'ABCL', 'ABCS', 'ABDC.1', 'ABEQ', 'ABMD', 'ABNB', 'ABOS', 'ABSI', 'ABVC', 'ABVX', 'ABXXF', 'ACAVF', 'ACBI', 'ACCD', 'ACDC', 'ACEL', 'ACER', 'ACET', 'ACETQ', 'ACEXF', 'ACGLN', 'ACGLP', 'ACGN', 'ACHL', 'ACHR', 'ACI', 'ACIC.1', 'ACIC.WS', 'ACIO', 'ACLH', 'ACLX', 'ACON', 'ACRGF', 'ACRHF', 'ACRV', 'ACT', 'ACTCW', 'ACTV', 'ACVA', 'ACVF', 'ACXP', 'ADC.PA', 'ADCT', 'ADEC', 'ADEXF', 'ADIV', 'ADLS', 'ADMS', 'ADN', 'ADOCU', 'ADOM.1', 'ADPT', 'ADPV', 'ADRO', 'ADSW', 'ADTX', 'ADV', 'ADVE', 'ADXS.1', 'AEB', 'AEBLF', 'AEFC', 'AEI', 'AEPPL', 'AEPPZ', 'AERI', 'AERO', 'AESI', 'AETH', 'AETI', 'AEVA', 'AFC', 'AFCL', 'AFGC', 'AFGE.1', 'AFGH', 'AFIB', 'AFIIQ', 'AFLG', 'AFMC', 'AFMCF', 'AFRM', 'AFSM', 'AFYA', 'AGAE', 'AGCB', 'AGCUU', 'AGFS', 'AGFY', 'AGGE', 'AGGH', 'AGGP

There are 5163 unique tickers to remove due to insufficient timestamps
There are 5082 tickers
Confirm there are no more columns with missing data Index([], dtype='object')
Removed 2784 tickers due to low dollar volume
There are 2298 tickers


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,ret_d,TBill3m,excess_ret_d,rel_ret_d
0,2019-01-02,A,AGILENT TECHNOLOGIES INC,318533000.0,2078511.0,65.69,66.5700,65.300,66.50,1.382210e+08,-0.021975,0.000096,-0.022071,-0.024338
1,2019-01-03,A,AGILENT TECHNOLOGIES INC,318533000.0,5383926.0,63.27,65.7800,62.000,65.53,3.528087e+08,0.024185,0.000095,0.024090,0.000941
2,2019-01-04,A,AGILENT TECHNOLOGIES INC,318533000.0,3119738.0,65.46,65.9500,64.090,64.09,1.999440e+08,0.029707,0.000094,0.029613,0.009853
3,2019-01-07,A,AGILENT TECHNOLOGIES INC,318533000.0,3221344.0,66.85,67.4250,65.610,65.64,2.114490e+08,0.009025,0.000095,0.008930,0.000826
4,2019-01-08,A,AGILENT TECHNOLOGIES INC,318533000.0,1578055.0,67.83,68.2100,66.700,67.59,1.066607e+08,0.012463,0.000095,0.012368,0.013571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132169,2024-05-24,ZYME,ZYMEWORKS INC,70705000.0,310059.0,8.72,8.9000,8.620,8.72,2.703714e+06,-0.029545,0.000208,-0.029754,-0.012262
3132170,2024-05-28,ZYME,ZYMEWORKS INC,70705000.0,319136.0,8.71,8.8900,8.635,8.80,2.808397e+06,-0.010539,0.000208,-0.010747,-0.010960
3132171,2024-05-29,ZYME,ZYMEWORKS INC,70705000.0,314990.0,8.48,8.5500,8.420,8.54,2.690015e+06,-0.014201,0.000208,-0.014409,-0.022240
3132172,2024-05-30,ZYME,ZYMEWORKS INC,70705000.0,284470.0,8.43,8.5500,8.390,8.45,2.403772e+06,0.000000,0.000208,-0.000208,0.000000


In [4]:
data = remove_dead_stocks(data)

data = portfolio(data, quantiles = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

factors = ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol']
cs_factors = ['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod']

data, factors, cs_factors = feature_engineer(data, factors, cs_factors)
print(f'Confirm that feature engineering did not create NaNs: {data.columns[data.isnull().sum() != 0]}')
print(f'All cs_factors: {cs_factors}')
print(f'All factors: {factors}')

data, num_of_tokens = make_sic_column(data, sic_code_path)
print(f'Confirm that adding the SIC column did not create NaNs: {data.columns[data.isnull().sum() != 0]}')

# Create dictionary that associate each ticker with a numerical label and vice versa for easier reference
num_to_tic_dict, tic_to_num_dict = num_tic_dicts(data)

data

NSTGQ is dead after 2024-02-07T00:00:00.000000000
SRNE is dead after 2023-09-28T00:00:00.000000000
SUNWQ is dead after 2024-02-06T00:00:00.000000000
ZOM is dead after 2020-09-18T00:00:00.000000000
Confirm that feature engineering did not create NaNs: Index([], dtype='object')
All cs_factors: ['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line']
All factors: ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line']
Confirm that adding the SIC column did not create NaNs: Index([], dtype='object')


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,close/MA10,close/MA5,open/MA10,open/MA5,STD_10day,H-L,RSI,MACD,MACD_Signal_Line,sic
0,2019-01-02,A,AGILENT TECHNOLOGIES INC,318533000.0,2078511.0,65.69,66.5700,65.300,66.50,1.382210e+08,...,0.987820,0.987820,1.000000,1.000000,0.000000,1.2700,53.855716,0.000000,0.000000,0.0
1,2019-01-03,A,AGILENT TECHNOLOGIES INC,318533000.0,5383926.0,63.27,65.7800,62.000,65.53,3.528087e+08,...,0.958419,0.958419,0.992653,0.992653,0.685894,3.7800,53.855716,-0.077379,-0.015476,0.0
2,2019-01-04,A,AGILENT TECHNOLOGIES INC,318533000.0,3119738.0,65.46,65.9500,64.090,64.09,1.999440e+08,...,1.001326,1.001326,0.980369,0.980369,1.212614,1.8600,53.855716,-0.251993,-0.062779,0.0
3,2019-01-07,A,AGILENT TECHNOLOGIES INC,318533000.0,3221344.0,66.85,67.4250,65.610,65.64,2.114490e+08,...,1.021546,1.021546,1.003056,1.003056,0.999033,1.8150,39.141414,-0.262281,-0.102680,0.0
4,2019-01-08,A,AGILENT TECHNOLOGIES INC,318533000.0,1578055.0,67.83,68.2100,66.700,67.59,1.066607e+08,...,1.029756,1.029756,1.026112,1.026112,1.293464,1.5100,59.221658,-0.111797,-0.104503,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132169,2024-05-24,ZYME,ZYMEWORKS INC,70705000.0,310059.0,8.72,8.9000,8.620,8.72,2.703714e+06,...,0.962685,0.974737,0.962685,0.974737,0.237197,0.2800,42.574257,-0.083236,-0.103528,10.0
3132170,2024-05-28,ZYME,ZYMEWORKS INC,70705000.0,319136.0,8.71,8.8900,8.635,8.80,2.808397e+06,...,0.960732,0.979973,0.970660,0.990099,0.225596,0.2550,46.283784,-0.092759,-0.101374,10.0
3132171,2024-05-29,ZYME,ZYMEWORKS INC,70705000.0,314990.0,8.48,8.5500,8.420,8.54,2.690015e+06,...,0.944005,0.962761,0.950685,0.969573,0.252413,0.1300,41.455696,-0.119903,-0.105080,10.0
3132172,2024-05-30,ZYME,ZYMEWORKS INC,70705000.0,284470.0,8.43,8.5500,8.390,8.45,2.403772e+06,...,0.946022,0.968743,0.948266,0.971041,0.292630,0.1600,41.194969,-0.146983,-0.113460,10.0


In [5]:
# Remove MACD_Signal_Line because it is highly correlated with MACD
factors.remove('MACD_Signal_Line')
cs_factors.remove('MACD_Signal_Line')

In [6]:
data = data.groupby("tic").apply(lambda x: add_all_ta_features(x, open="prccd", high="prchd", low="prcld", close="prcod", volume="cshtrd", fillna=True))
data = data.reset_index(level=0, drop=True).reset_index(drop=True)
data

C:\Users\rguo\AppData\Local\Temp\ipykernel_15616\1178857726.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby("tic").apply(lambda x: add_all_ta_features(x, open="prccd", high="prchd", low="prcld", close="prcod", volume="cshtrd", fillna=True))


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2019-01-02,A,AGILENT TECHNOLOGIES INC,318533000.0,2078511.0,65.69,66.5700,65.300,66.50,1.382210e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66.500000,0.000000,0.000000,0.000000
1,2019-01-03,A,AGILENT TECHNOLOGIES INC,318533000.0,5383926.0,63.27,65.7800,62.000,65.53,3.528087e+08,...,-0.116485,-0.023297,-0.093188,11.349090,2.269818,9.079272,65.752362,-1.458647,-1.469389,-1.458647
2,2019-01-04,A,AGILENT TECHNOLOGIES INC,318533000.0,3119738.0,65.46,65.9500,64.090,64.09,1.999440e+08,...,-0.380339,-0.094705,-0.285634,12.031942,4.222243,7.809699,64.474366,-2.197467,-2.221971,-3.624060
3,2019-01-07,A,AGILENT TECHNOLOGIES INC,318533000.0,3221344.0,66.85,67.4250,65.610,65.64,2.114490e+08,...,-0.396139,-0.154992,-0.241147,12.659965,5.909787,6.750178,65.292937,2.418474,2.389692,-1.293233
4,2019-01-08,A,AGILENT TECHNOLOGIES INC,318533000.0,1578055.0,67.83,68.2100,66.700,67.59,1.066607e+08,...,-0.168593,-0.157712,-0.010881,8.097391,6.347308,1.750083,66.728041,2.970750,2.927478,1.639098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128080,2024-05-24,ZYME,ZYMEWORKS INC,70705000.0,310059.0,8.72,8.9000,8.620,8.72,2.703714e+06,...,-0.916710,-1.130908,0.214197,-3.714546,-0.715430,-2.999116,8.949957,-3.111111,-3.160534,-41.866667
3128081,2024-05-28,ZYME,ZYMEWORKS INC,70705000.0,319136.0,8.71,8.8900,8.635,8.80,2.808397e+06,...,-1.023920,-1.109510,0.085590,-5.393208,-1.650985,-3.742223,8.948779,0.917431,0.913248,-41.333333
3128082,2024-05-29,ZYME,ZYMEWORKS INC,70705000.0,314990.0,8.48,8.5500,8.420,8.54,2.690015e+06,...,-1.329194,-1.153447,-0.175747,-6.797857,-2.680360,-4.117498,8.891313,-2.954545,-2.999071,-43.066667
3128083,2024-05-30,ZYME,ZYMEWORKS INC,70705000.0,284470.0,8.43,8.5500,8.390,8.45,2.403772e+06,...,-1.637062,-1.250170,-0.386892,-8.520380,-3.848364,-4.672016,8.836150,-1.053864,-1.059457,-43.666667


In [7]:
TA_factors = [# Momentum indicators
              'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr',
              # Volume indicators
              'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt',
              # Volatility indicators
              'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui',
              # Trend indicators
              'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a',
              # Other indicators
              'others_dr'
]

factors.extend(TA_factors)
cs_factors.extend(TA_factors)

print(factors)
print(cs_factors)

data = data[['datadate', 'tic', 'conm'] + factors + ['ret_d', 'TBill3m', 'excess_ret_d', 'rel_ret_d', 'DistinctRank', 'rank', 'sic']]
data

['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr', 'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt', 'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui', 'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a', 'others_dr']
['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr', 'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt', 'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility

,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
0,2019-01-02,A,AGILENT TECHNOLOGIES INC,318533000.0,2078511.0,65.69,66.5700,65.300,66.50,1.382210e+08,...,0.0,65.935000,0.000000,-0.021975,0.000096,-0.022071,-0.024338,201.0,-2,0.0
1,2019-01-03,A,AGILENT TECHNOLOGIES INC,318533000.0,5383926.0,63.27,65.7800,62.000,65.53,3.528087e+08,...,4.0,64.285000,-1.458647,0.024185,0.000095,0.024090,0.000941,1399.0,1,0.0
2,2019-01-04,A,AGILENT TECHNOLOGIES INC,318533000.0,3119738.0,65.46,65.9500,64.090,64.09,1.999440e+08,...,4.0,64.285000,-2.197467,0.029707,0.000094,0.029613,0.009853,1708.0,1,0.0
3,2019-01-07,A,AGILENT TECHNOLOGIES INC,318533000.0,3221344.0,66.85,67.4250,65.610,65.64,2.114490e+08,...,4.0,64.712500,2.418474,0.009025,0.000095,0.008930,0.000826,1268.0,0,0.0
4,2019-01-08,A,AGILENT TECHNOLOGIES INC,318533000.0,1578055.0,67.83,68.2100,66.700,67.59,1.066607e+08,...,4.0,65.105000,2.970750,0.012463,0.000095,0.012368,0.013571,1950.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128080,2024-05-24,ZYME,ZYMEWORKS INC,70705000.0,310059.0,8.72,8.9000,8.620,8.72,2.703714e+06,...,20.0,8.957350,-3.111111,-0.029545,0.000208,-0.029754,-0.012262,457.0,-2,10.0
3128081,2024-05-28,ZYME,ZYMEWORKS INC,70705000.0,319136.0,8.71,8.8900,8.635,8.80,2.808397e+06,...,16.0,8.943675,0.917431,-0.010539,0.000208,-0.010747,-0.010960,411.0,-2,10.0
3128082,2024-05-29,ZYME,ZYMEWORKS INC,70705000.0,314990.0,8.48,8.5500,8.420,8.54,2.690015e+06,...,12.0,8.862425,-2.954545,-0.014201,0.000208,-0.014409,-0.022240,163.0,-2,10.0
3128083,2024-05-30,ZYME,ZYMEWORKS INC,70705000.0,284470.0,8.43,8.5500,8.390,8.45,2.403772e+06,...,8.0,8.854925,-1.053864,0.000000,0.000208,-0.000208,0.000000,2298.0,2,10.0


In [8]:
# if 'datadate' not in cs_factors:
#     cs_factors.append('datadate')
# if 'tic' not in tic_factors:
#     tic_factors.append('tic')
# data = standardize_data(data, cs_factors)
# data

In [9]:
data.to_csv('processed_data_20190101-20240531_40factors.csv')

In [10]:
# Save 20 factors data
data_20factors = data[['datadate', 'tic', 'conm'] + list(set(factors) - set(TA_factors)) + ['ret_d', 'TBill3m', 'excess_ret_d', 'rel_ret_d', 'DistinctRank', 'rank', 'sic']]
data_20factors

,datadate,tic,conm,H-L,open/MA10,close/MA5,dol_vol,cshtrd,MA_5day,prcld,...,Mom_2day,RSI,Mom_3day,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
0,2019-01-02,A,AGILENT TECHNOLOGIES INC,-0.037289,0.000000,-1.060324,0.002255,-0.025221,-0.034278,-0.033713,...,0.000000,0.829033,0.000000,-0.021975,0.000096,-0.022071,-0.024338,201.0,-2,0.0
1,2019-01-03,A,AGILENT TECHNOLOGIES INC,-0.033321,-0.927726,-1.444869,0.223431,-0.022752,-0.034258,-0.034721,...,0.000000,-1.460064,0.000000,0.024185,0.000095,0.024090,0.000941,1399.0,1,0.0
2,2019-01-04,A,AGILENT TECHNOLOGIES INC,-0.036410,-1.364040,-0.845772,0.046170,-0.027714,-0.034402,-0.034075,...,-1.281771,-0.724524,0.000000,0.029707,0.000094,0.029613,0.009853,1708.0,1,0.0
3,2019-01-07,A,AGILENT TECHNOLOGIES INC,-0.036418,-0.587161,-0.275395,0.096857,-0.028976,-0.034374,-0.033867,...,-0.497024,-1.736556,-0.844101,0.009025,0.000095,0.008930,0.000826,1268.0,0,0.0
4,2019-01-08,A,AGILENT TECHNOLOGIES INC,-0.037258,-0.182538,-0.153371,-0.051684,-0.028080,-0.034278,-0.033823,...,0.172805,-1.046498,-0.231378,0.012463,0.000095,0.012368,0.013571,1950.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3128080,2024-05-24,ZYME,ZYMEWORKS INC,-0.038249,-0.666867,-0.539314,-0.152072,-0.279985,-0.028191,-0.028215,...,-0.572051,-0.609704,-0.233830,-0.029545,0.000208,-0.029754,-0.012262,457.0,-2,10.0
3128081,2024-05-28,ZYME,ZYMEWORKS INC,-0.029917,-0.555507,-0.293324,-0.133677,-0.338022,-0.028214,-0.028255,...,-0.465558,-0.352282,-0.339845,-0.010539,0.000208,-0.010747,-0.010960,411.0,-2,10.0
3128082,2024-05-29,ZYME,ZYMEWORKS INC,-0.033637,-0.683660,-0.625479,-0.143290,-0.330455,-0.028218,-0.028187,...,-0.278332,-0.430958,-0.675574,-0.014201,0.000208,-0.014409,-0.022240,163.0,-2,10.0
3128083,2024-05-30,ZYME,ZYMEWORKS INC,-0.029847,-0.847794,-0.761193,-0.153272,-0.352201,-0.028228,-0.028180,...,-0.778909,-0.370356,-0.456588,0.000000,0.000208,-0.000208,0.000000,2298.0,2,10.0


In [11]:
data_20factors.to_csv('processed_data_20190101-20240531_20factors.csv')

In [14]:
data['ret_d'].mean()

0.0006426119253910602

In [15]:
data['ret_d'].std()

0.03147183387322557

In [12]:
data['ret_d'].max()

5.381766381766382

In [13]:
data['ret_d'].min()

-0.8957877721783779

In [18]:
data[data['ret_d'] < -0.5]

,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
87723,2020-12-11,AIV,APARTMENT INVST & MGMT CO,-0.033211,-0.038724,-0.028201,-0.028302,-0.028126,-0.028241,-0.152637,...,-0.436814,-0.028355,-0.377143,-0.856708,0.000003,-0.856711,-0.850545,1.0,-2,8.0
105699,2021-12-20,ALLK,ALLAKOS INC,-0.047410,-0.032027,-0.022995,-0.022947,-0.023076,-0.022992,-0.139785,...,-0.298085,-0.023598,0.577392,-0.871383,0.000002,-0.871385,-0.885560,1.0,-2,10.0
106216,2024-01-11,ALLK,ALLAKOS INC,-0.169381,-0.126092,-0.028975,-0.028925,-0.028892,-0.028882,-0.168162,...,-0.776181,-0.028972,-1.215084,-0.601307,0.000207,-0.601515,-0.585354,1.0,-2,10.0
223390,2023-11-07,ATRA,ATARA BIOTHERAPEUTICS INC,0.786715,1.135056,-0.025762,-0.025625,-0.025777,-0.025783,-0.179343,...,1.034474,-0.025478,-0.049567,-0.638000,0.000208,-0.638208,-0.635058,1.0,-2,10.0
263907,2022-05-13,AZPN,ASPEN TECHNOLOGY INC,-0.031358,-0.023823,0.010983,0.011004,0.010419,0.010532,-0.140478,...,-1.121170,0.008438,0.204857,-0.536036,0.000037,-0.536074,-0.551262,1.0,-2,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829899,2020-03-27,VANI,VIVANI MEDICAL INC,-0.035824,-0.066414,-0.034591,-0.034506,-0.034308,-0.034466,-0.158656,...,0.739361,-0.035433,-3.000000,-0.637168,-0.000002,-0.637166,-0.644540,1.0,-2,15.0
2892581,2020-03-05,VNOM,VIPER ENERGY INC,-0.035100,-0.043528,-0.032482,-0.032616,-0.032248,-0.032266,-0.156727,...,0.830377,-0.032011,-0.758386,-0.511301,0.000029,-0.511330,-0.445985,6.0,-2,301.0
2957775,2019-04-05,VXRT,VAXART INC,-0.043557,-0.031575,-0.040271,-0.040424,-0.040101,-0.040243,-0.242037,...,-0.896856,-0.040808,-1.092096,-0.505208,0.000095,-0.505304,-0.503521,1.0,-2,10.0
2971030,2023-03-09,WAL,WESTERN ALLIANCE BANCORP,-0.150775,-0.026935,-0.023278,-0.022667,-0.023346,-0.022613,0.081755,...,0.917365,-0.022697,-1.489510,-0.764351,0.000194,-0.764545,-0.730395,1.0,-2,11.0


# 2006-2010 data

In [19]:
# For importing pre-processed data and further process here
data_path = 'C:/Projects/Stock_price_prediction/Experiments/WRDS_raw_20060101-20101231.csv'
TBill_path = 'C:/Projects/Stock_price_prediction/3monthTBill_2006-03-01_2024-07-12.csv'
sic_code_path = 'C:/Projects/Stock_price_prediction/Experiments/tickers&sics_20060101-20101231.csv'

In [20]:
data = pd.read_csv(data_path)

data['datadate'] = pd.to_datetime(data['datadate'], format='%Y-%m-%d')
data = data.sort_values(by=['tic', 'datadate']).reset_index(drop=True)

total_dates = set(data['datadate'].unique())
apple_dates = set(data[data['tic']=='AAPL']['datadate'].unique())
print(len(apple_dates))
print(len(total_dates - apple_dates))

# Uses the numbers of days for Apple as the standard and drop any other stock with fewer dates
num_of_ts = len(data[data['tic']=='AAPL']['datadate'].unique())
factors = ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod']
print(f'There are {num_of_ts} timestamps')

tickers = list(data.tic.unique())
print(f'There are {len(tickers)} tickers')

# Removes stocks with missing data
data, tickers = remove_tic(data, tickers, num_of_ts, factors)

# Check that each ticker has exactly num_of_ts timestamps
assert data.shape[0] == num_of_ts * len(tickers)

data, tickers = remove_low_dollar_vol(data, tickers, dol_vol_thres = 10000000)

data = data.reset_index(drop=True)
data, tickers = adjust_split(data, tickers, factors)

data = compute_ret(data, TBill_path)
data

1259
13
There are 1259 timestamps
There are 19504 tickers
Removed ['3408B', '3LGCFF', '3RELLB', '6189B', 'AABC', 'AACAY', 'ABCZY', 'ACAVF', 'ACCI', 'ACCYY', 'ACEAF', 'ADPAJ', 'ADPAS', 'ADRI', 'AFK.2', 'AGGZF', 'AIV.PG', 'ALF.1', 'ALPMY', 'ALSMY', 'ALTM.1', 'AMXX', 'AMY30', 'AMY40', 'AMYRZ', 'ANPDY', 'AORGA', 'AORGB', 'ASBDF', 'ASGLY', 'ASIL', 'ATISZ', 'ATLKY', 'AURRF', 'AVNDF', 'AVSS', 'AVY.PA', 'BACHY', 'BBAVY', 'BCT', 'BDGAB', 'BDRBF', 'BFC.3', 'BFRFF', 'BGO.1', 'BHC.P', 'BKHYY', 'BKKLY', 'BLKCF', 'BLKIA', 'BLN.2', 'BNEFF', 'BOWFF', 'BRBMF', 'BRDCY', 'BRF.1', 'BRRAY', 'BTE', 'BURCB', 'BWC.1', 'BYL.P', 'C.PH.1', 'CBAUY', 'CBCY', 'CBCYB', 'CBL.PB', 'CERB', 'CERBB', 'CEY.3', 'CFHZ', 'CHEOY', 'CHHM', 'CHM.U', 'CLP.PD', 'CLP.PE', 'CMBC', 'CMM.PB', 'CMNFY', 'CMNR', 'CNBKB', 'CNE.4', 'CNIRF', 'CNT.PB', 'CNVLZ', 'COKEB', 'CPM.1', 'CPQRF', 'CRBJY', 'CRLTS', 'CRNS', 'CRWFF', 'CSPLF', 'CUKPF', 'CUPUF', 'CZBC', 'DANKY', 'DAYYF', 'DKI', 'DKK', 'DKL.1', 'DKR.', 'DKW', 'DKY', 'DLR.PA', 'DLR.PB', 'D

Removed ['3CYBR', '3DLNKQ', '3ESPIQ', '3HSRC', '3JPEIE', '3MAMM', '3MAWI', '3SAZZ', '3TWAIQ', '3UCPC', '3USWI', '3VIPPS', '3ZARLF', '6911B', '7686B.1', 'AA.P', 'AAAGY', 'AAC.1', 'AACB', 'AACG', 'AACH', 'AACOU.1', 'AACOW.1', 'AACPF', 'AACS', 'AADG', 'AADI.1', 'AADR', 'AAFGQ', 'AAGC', 'AAGH.1', 'AAIIQ', 'AAIR', 'AAMA', 'AAMC.', 'AAME', 'AAMUQ', 'AANB', 'AANI', 'AAPH', 'AAPI', 'AAPT', 'AASP', 'AATC', 'AATRL', 'AATV', 'AAUKF', 'AAWW', 'AAXJ', 'AAY', 'ABAT.1', 'ABBB', 'ABBC', 'ABCC', 'ABCD', 'ABCP', 'ABDR', 'ABDS', 'ABEO', 'ABEV', 'ABEW', 'ABFIQ', 'ABI', 'ABIO.1', 'ABIX', 'ABKH', 'ABKI', 'ABKOQ', 'ABLE', 'ABLT', 'ABMC', 'ABMT', 'ABNJ', 'ABNK', 'ABNS', 'ABNYY', 'ABOYY', 'ABPH', 'ABSI.1', 'ABTI.1', 'ABVA', 'ABVC.1', 'ABVT', 'ABVV', 'ABW.PA', 'ABWG', 'ABY.3', 'ACAI', 'ACAMY', 'ACAR.', 'ACARQ', 'ACB.1', 'ACBA.1', 'ACBCQ', 'ACCA', 'ACCS.', 'ACCY', 'ACDQ', 'ACDU', 'ACEL.', 'ACEN', 'ACEXF', 'ACFC', 'ACFL', 'ACFN', 'ACGBY', 'ACGI', 'ACGJ', 'ACGN', 'ACGX', 'ACHHY', 'ACHN.1', 'ACIC', 'ACIE', 'ACII.1'

There are 12388 unique tickers to remove due to missing values
Removed ['3ACCO', '3AGROE', '3BBTCE', '3CLPTQ', '3CSPNE', '3DSCSE', '3FRES', '3HCISQ', '3INWN', '3LOCKE', '3MBRWQ', '3PGMTE', '3RCLAE', '3RGEYE', '3STGM', '3WDSI', 'AAALF', 'AACE.', 'AAIIQ.1', 'AAMTF', 'AAN.2', 'AATK', 'ABA', 'ABCRQ', 'ABGX', 'ABI.3', 'ABII', 'ABIO', 'ABLP', 'ABLSQ', 'ABLZF', 'ABMIQ', 'ABS.1', 'ABSO', 'ABTI.', 'ABTXQ', 'ABUS', 'ABWTQ', 'ABY.2', 'ACAP.1', 'ACCN', 'ACE.PC', 'ACERF', 'ACF', 'ACHFF', 'ACLI', 'ACM', 'ACMP', 'ACNT.', 'ACOM', 'ACR', 'ACR.1', 'ACS', 'ACTI', 'ACTL.1', 'ACWI', 'AD', 'ADBL', 'ADCO', 'ADCT.1', 'ADEX.', 'ADF.1', 'ADIC', 'ADM.PA', 'ADPAN', 'ADRX', 'ADXDF', 'ADZA', 'AEENQ', 'AEF.1', 'AEF.3', 'AEGR', 'AEGZF', 'AEIR', 'AEMLW', 'AENYQ', 'AER', 'AEV', 'AEXAF', 'AEZ', 'AFBR', 'AFC', 'AFCL', 'AFCO.1', 'AFF', 'AFFY', 'AFQ', 'AFR', 'AFRAF', 'AG', 'AGAM', 'AGC.2', 'AGCCQ', 'AGD', 'AGE.1', 'AGIXQ', 'AGL.1', 'AGNC', 'AGPT', 'AGR.3', 'AH.Z', 'AHEXF', 'AHFI', 'AHG.', 'AHGP', 'AHODF', 'AHWYQ', 'AIC.3',

There are 3406 unique tickers to remove due to insufficient timestamps
There are 3710 tickers
Confirm there are no more columns with missing data Index(['cshoc'], dtype='object')
Removed 2169 tickers due to low dollar volume
There are 1541 tickers


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,ret_d,TBill3m,excess_ret_d,rel_ret_d
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,513000000.0,3796200.0,33.50,33.58,32.8151,33.40,1.267931e+08,-0.002981,0.000000,-0.002981,-0.011364
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,513000000.0,3001300.0,33.59,33.83,33.3700,33.55,1.006936e+08,0.031390,0.000000,0.031390,0.023674
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,513000000.0,3458800.0,34.47,34.47,33.4500,33.45,1.156969e+08,0.004348,0.000000,0.004348,-0.001524
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,513000000.0,4396500.0,34.65,34.79,34.0900,34.50,1.516792e+08,-0.011544,0.000000,-0.011544,-0.013951
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,513000000.0,2920500.0,34.55,34.80,34.4200,34.65,1.011953e+08,0.021898,0.000000,0.021898,0.014033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940114,2010-12-27,ZUMZ,ZUMIEZ INC,30669000.0,260462.0,28.83,29.14,28.0000,28.50,7.423167e+06,-0.007610,0.000006,-0.007616,-0.006422
1940115,2010-12-28,ZUMZ,ZUMIEZ INC,30669000.0,287528.0,28.65,29.28,28.5700,28.91,8.312434e+06,-0.020565,0.000006,-0.020571,-0.021271
1940116,2010-12-29,ZUMZ,ZUMIEZ INC,30669000.0,227262.0,28.21,28.98,28.1000,28.69,6.520147e+06,-0.013523,0.000006,-0.013529,-0.013398
1940117,2010-12-30,ZUMZ,ZUMIEZ INC,30669000.0,221368.0,27.71,28.72,27.6900,28.10,6.220441e+06,0.000000,0.000006,-0.000006,0.000000


In [21]:
data = remove_dead_stocks(data)

data = portfolio(data, quantiles = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0])

factors = ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol']
cs_factors = ['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod']

data, factors, cs_factors = feature_engineer(data, factors, cs_factors)
print(f'Confirm that feature engineering did not create NaNs: {data.columns[data.isnull().sum() != 0]}')
print(f'All cs_factors: {cs_factors}')
print(f'All factors: {factors}')

data, num_of_tokens = make_sic_column(data, sic_code_path)
print(f'Confirm that adding the SIC column did not create NaNs: {data.columns[data.isnull().sum() != 0]}')

# Create dictionary that associate each ticker with a numerical label and vice versa for easier reference
num_to_tic_dict, tic_to_num_dict = num_tic_dicts(data)

data

IDMCQ is dead after 2008-07-15T00:00:00.000000000
LEHMQ is dead after 2008-10-10T00:00:00.000000000
NRTLQ is dead after 2009-01-14T00:00:00.000000000
SIRI is dead after 2009-02-11T00:00:00.000000000
THMRQ is dead after 2009-02-02T00:00:00.000000000
WAMUQ is dead after 2008-09-29T00:00:00.000000000
Confirm that feature engineering did not create NaNs: Index([], dtype='object')
All cs_factors: ['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line']
All factors: ['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'MACD_Signal_Line']
Confirm that adding the SIC column did not create NaNs: Index([], dtype='object')


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,close/MA10,close/MA5,open/MA10,open/MA5,STD_10day,H-L,RSI,MACD,MACD_Signal_Line,sic
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,513000000.0,3796200.0,33.50,33.58,32.8151,33.40,1.267931e+08,...,1.002994,1.002994,1.000000,1.000000,0.000000,0.7649,53.401349,0.000000,0.000000,0.0
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,513000000.0,3001300.0,33.59,33.83,33.3700,33.55,1.006936e+08,...,1.003435,1.003435,1.002240,1.002240,0.106066,0.4600,100.000000,0.011966,0.002393,0.0
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,513000000.0,3458800.0,34.47,34.47,33.4500,33.45,1.156969e+08,...,1.029980,1.029980,0.999502,0.999502,0.076376,1.0200,60.000000,0.013227,0.004560,0.0
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,513000000.0,4396500.0,34.65,34.79,34.0900,34.50,1.516792e+08,...,1.027428,1.027428,1.022980,1.022980,0.520416,0.7000,92.307692,0.097825,0.023213,0.0
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,513000000.0,2920500.0,34.55,34.80,34.4200,34.65,1.011953e+08,...,1.018873,1.018873,1.021822,1.021822,0.611760,0.3800,93.103448,0.174957,0.053562,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940114,2010-12-27,ZUMZ,ZUMIEZ INC,30669000.0,260462.0,28.83,29.14,28.0000,28.50,7.423167e+06,...,0.984026,0.992427,0.972763,0.981067,0.564108,1.1400,38.364780,0.120371,0.466884,59.0
1940115,2010-12-28,ZUMZ,ZUMIEZ INC,30669000.0,287528.0,28.65,29.28,28.5700,28.91,8.312434e+06,...,0.981501,0.989569,0.990408,0.998549,0.518438,0.7100,35.420099,0.077767,0.389061,59.0
1940116,2010-12-29,ZUMZ,ZUMIEZ INC,30669000.0,227262.0,28.21,28.98,28.1000,28.69,6.520147e+06,...,0.970383,0.981491,0.986894,0.998191,0.477411,0.8800,29.351536,0.025951,0.316439,59.0
1940117,2010-12-30,ZUMZ,ZUMIEZ INC,30669000.0,221368.0,27.71,28.72,27.6900,28.10,6.220441e+06,...,0.955913,0.971599,0.969367,0.985273,0.568171,1.0300,31.386861,-0.062007,0.240750,59.0


In [22]:
# Remove MACD_Signal_Line because it is highly correlated with MACD
factors.remove('MACD_Signal_Line')
cs_factors.remove('MACD_Signal_Line')

In [23]:
data = data.groupby("tic").apply(lambda x: add_all_ta_features(x, open="prccd", high="prchd", low="prcld", close="prcod", volume="cshtrd", fillna=True))
data = data.reset_index(level=0, drop=True).reset_index(drop=True)
data

C:\Users\rguo\AppData\Local\Temp\ipykernel_40868\1178857726.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby("tic").apply(lambda x: add_all_ta_features(x, open="prccd", high="prchd", low="prcld", close="prcod", volume="cshtrd", fillna=True))


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,513000000.0,3796200.0,33.50,33.58,32.8151,33.40,1.267931e+08,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,33.400000,0.000000,0.000000,0.000000
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,513000000.0,3001300.0,33.59,33.83,33.3700,33.55,1.006936e+08,...,0.035814,0.007163,0.028651,-1.696693,-0.339339,-1.357354,33.444829,0.449102,0.448096,0.449102
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,513000000.0,3458800.0,34.47,34.47,33.4500,33.45,1.156969e+08,...,0.039586,0.013647,0.025938,-2.041423,-0.679756,-1.361668,33.446617,-0.298063,-0.298508,0.149701
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,513000000.0,4396500.0,34.65,34.79,34.0900,34.50,1.516792e+08,...,0.292064,0.069331,0.222734,-0.264675,-0.596740,0.332064,33.671531,3.139013,3.090754,3.293413
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,513000000.0,2920500.0,34.55,34.80,34.4200,34.65,1.011953e+08,...,0.521015,0.159668,0.361347,-2.050761,-0.887544,-1.163217,33.880505,0.434783,0.433840,3.742515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938855,2010-12-27,ZUMZ,ZUMIEZ INC,30669000.0,260462.0,28.83,29.14,28.0000,28.50,7.423167e+06,...,0.412061,1.605160,-1.193099,-6.869538,0.694436,-7.563973,29.512614,0.352113,0.351494,30.614115
1938856,2010-12-28,ZUMZ,ZUMIEZ INC,30669000.0,287528.0,28.65,29.28,28.5700,28.91,8.312434e+06,...,0.266419,1.337412,-1.070993,-7.484235,-0.941298,-6.542936,29.486121,1.438596,1.428347,32.493126
1938857,2010-12-29,ZUMZ,ZUMIEZ INC,30669000.0,227262.0,28.21,28.98,28.1000,28.69,6.520147e+06,...,0.089018,1.087733,-0.998715,-9.413118,-2.635662,-6.777456,29.447368,-0.760982,-0.763893,31.484876
1938858,2010-12-30,ZUMZ,ZUMIEZ INC,30669000.0,221368.0,27.71,28.72,27.6900,28.10,6.220441e+06,...,-0.213267,0.827533,-1.040800,-11.087133,-4.325956,-6.761177,29.402409,-2.056466,-2.077905,28.780935


In [24]:
TA_factors = [# Momentum indicators
              'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr',
              # Volume indicators
              'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt',
              # Volatility indicators
              'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui',
              # Trend indicators
              'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a',
              # Other indicators
              'others_dr'
]

factors.extend(TA_factors)
cs_factors.extend(TA_factors)

print(factors)
print(cs_factors)

data = data[['datadate', 'tic', 'conm'] + factors + ['ret_d', 'TBill3m', 'excess_ret_d', 'rel_ret_d', 'DistinctRank', 'rank', 'sic']]
data

['cshoc', 'cshtrd', 'prccd', 'prchd', 'prcld', 'prcod', 'dol_vol', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr', 'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt', 'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility_ui', 'trend_adx', 'trend_aroon_up', 'trend_aroon_down', 'trend_ichimoku_a', 'others_dr']
['cshoc', 'cshtrd', 'datadate', 'dol_vol', 'prccd', 'prchd', 'prcld', 'prcod', 'Mom_2day', 'Mom_3day', 'Mom_5day', 'MA_5day', 'MA_10day', 'close/MA10', 'close/MA5', 'open/MA10', 'open/MA5', 'STD_10day', 'H-L', 'RSI', 'MACD', 'momentum_stoch_rsi', 'momentum_stoch', 'momentum_ao', 'momentum_pvo', 'momentum_kama', 'momentum_wr', 'volume_adi', 'volume_em', 'volume_fi', 'volume_cmf', 'volume_vpt', 'volatility_atr', 'volatility_bbh', 'volatility_dcw', 'volatility

,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,513000000.0,3796200.0,33.50,33.58,32.8151,33.40,1.267931e+08,...,0.0,33.19755,0.000000,-0.002981,0.000000,-0.002981,-0.011364,357.0,-1,0.0
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,513000000.0,3001300.0,33.59,33.83,33.3700,33.55,1.006936e+08,...,0.0,33.32255,0.449102,0.031390,0.000000,0.031390,0.023674,1406.0,2,0.0
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,513000000.0,3458800.0,34.47,34.47,33.4500,33.45,1.156969e+08,...,0.0,33.64255,-0.298063,0.004348,0.000000,0.004348,-0.001524,797.0,0,0.0
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,513000000.0,4396500.0,34.65,34.79,34.0900,34.50,1.516792e+08,...,0.0,33.80255,3.139013,-0.011544,0.000000,-0.011544,-0.013951,243.0,-2,0.0
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,513000000.0,2920500.0,34.55,34.80,34.4200,34.65,1.011953e+08,...,0.0,33.80755,0.434783,0.021898,0.000000,0.021898,0.014033,1314.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938855,2010-12-27,ZUMZ,ZUMIEZ INC,30669000.0,260462.0,28.83,29.14,28.0000,28.50,7.423167e+06,...,100.0,29.78725,0.352113,-0.007610,0.000006,-0.007616,-0.006422,277.0,-2,59.0
1938856,2010-12-28,ZUMZ,ZUMIEZ INC,30669000.0,287528.0,28.65,29.28,28.5700,28.91,8.312434e+06,...,96.0,29.76225,1.438596,-0.020565,0.000006,-0.020571,-0.021271,23.0,-2,59.0
1938857,2010-12-29,ZUMZ,ZUMIEZ INC,30669000.0,227262.0,28.21,28.98,28.1000,28.69,6.520147e+06,...,92.0,29.76225,-0.760982,-0.013523,0.000006,-0.013529,-0.013398,67.0,-2,59.0
1938858,2010-12-30,ZUMZ,ZUMIEZ INC,30669000.0,221368.0,27.71,28.72,27.6900,28.10,6.220441e+06,...,100.0,29.58725,-2.056466,0.000000,0.000006,-0.000006,0.000000,1541.0,2,59.0


In [25]:
# if 'datadate' not in cs_factors:
#     cs_factors.append('datadate')
# if 'tic' not in tic_factors:
#     tic_factors.append('tic')
data = standardize_data(data, cs_factors)
data

Confirm that standardization did not create NaNs: Index([], dtype='object')


,datadate,tic,conm,cshoc,cshtrd,prccd,prchd,prcld,prcod,dol_vol,...,trend_aroon_down,trend_ichimoku_a,others_dr,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,-0.025595,-0.025630,-0.025569,-0.025569,-0.025568,-0.025568,0.106411,...,0.000000,-0.025569,0.000000,-0.002981,0.000000,-0.002981,-0.011364,357.0,-1,0.0
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,-0.025595,-0.025588,-0.025569,-0.025569,-0.025570,-0.025570,0.061284,...,-0.290607,-0.025569,-0.316155,0.031390,0.000000,0.031390,0.023674,1406.0,2,0.0
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,-0.025595,-0.025623,-0.025565,-0.025565,-0.025567,-0.025567,0.145104,...,-0.384219,-0.025567,-0.575402,0.004348,0.000000,0.004348,-0.001524,797.0,0,0.0
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,-0.025595,-0.025567,-0.025563,-0.025564,-0.025565,-0.025566,0.181500,...,-0.407270,-0.025566,1.259968,-0.011544,0.000000,-0.011544,-0.013951,243.0,-2,0.0
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,-0.025595,-0.025583,-0.025566,-0.025563,-0.025566,-0.025563,0.087128,...,-0.417315,-0.025566,-0.083690,0.021898,0.000000,0.021898,0.014033,1314.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938855,2010-12-27,ZUMZ,ZUMIEZ INC,-0.025493,-0.025483,-0.025545,-0.025541,-0.025545,-0.025540,-0.202324,...,2.855976,-0.025540,0.656636,-0.007610,0.000006,-0.007616,-0.006422,277.0,-2,59.0
1938856,2010-12-28,ZUMZ,ZUMIEZ INC,-0.025493,-0.025484,-0.025545,-0.025544,-0.025545,-0.025544,-0.227474,...,2.789317,-0.025541,0.633605,-0.020565,0.000006,-0.020571,-0.021271,23.0,-2,59.0
1938857,2010-12-29,ZUMZ,ZUMIEZ INC,-0.025493,-0.025484,-0.025546,-0.025545,-0.025546,-0.025545,-0.238048,...,2.710829,-0.025541,-0.527198,-0.013523,0.000006,-0.013529,-0.013398,67.0,-2,59.0
1938858,2010-12-30,ZUMZ,ZUMIEZ INC,-0.025493,-0.025484,-0.025547,-0.025546,-0.025547,-0.025546,-0.189909,...,3.000000,-0.025542,-1.690540,0.000000,0.000006,-0.000006,0.000000,1541.0,2,59.0


In [26]:
data.to_csv('processed_data_20060101-20101231_40factors.csv')

In [27]:
# Save 20 factors data
data_20factors = data[['datadate', 'tic', 'conm'] + list(set(factors) - set(TA_factors)) + ['ret_d', 'TBill3m', 'excess_ret_d', 'rel_ret_d', 'DistinctRank', 'rank', 'sic']]
data_20factors

,datadate,tic,conm,H-L,open/MA10,close/MA5,dol_vol,cshtrd,MA_5day,prcld,...,Mom_2day,RSI,Mom_3day,ret_d,TBill3m,excess_ret_d,rel_ret_d,DistinctRank,rank,sic
0,2006-01-03,A,AGILENT TECHNOLOGIES INC,-0.025580,0.000000,-0.367269,0.106411,-0.025630,-0.025568,-0.025568,...,0.000000,0.937261,0.000000,-0.002981,0.000000,-0.002981,-0.011364,357.0,-1,0.0
1,2006-01-04,A,AGILENT TECHNOLOGIES INC,-0.025551,-0.298387,-0.515824,0.061284,-0.025588,-0.025569,-0.025570,...,0.000000,0.594669,0.000000,0.031390,0.000000,0.031390,0.023674,1406.0,2,0.0
2,2006-01-05,A,AGILENT TECHNOLOGIES INC,-0.025511,-0.645923,0.891116,0.145104,-0.025623,-0.025568,-0.025567,...,-0.609305,-0.847549,0.000000,0.004348,0.000000,0.004348,-0.001524,797.0,0,0.0
3,2006-01-06,A,AGILENT TECHNOLOGIES INC,-0.025533,0.577031,0.406755,0.181500,-0.025567,-0.025568,-0.025565,...,0.450504,0.518398,0.133421,-0.011544,0.000000,-0.011544,-0.013951,243.0,-2,0.0
4,2006-01-09,A,AGILENT TECHNOLOGIES INC,-0.025505,0.344636,-0.107930,0.087128,-0.025583,-0.025567,-0.025566,...,0.851816,0.657036,0.320737,0.021898,0.000000,0.021898,0.014033,1314.0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938855,2010-12-27,ZUMZ,ZUMIEZ INC,-0.025483,-1.272655,-0.653945,-0.202324,-0.025483,-0.025540,-0.025545,...,-1.164320,-1.257952,-1.791900,-0.007610,0.000006,-0.007616,-0.006422,277.0,-2,59.0
1938856,2010-12-28,ZUMZ,ZUMIEZ INC,-0.025489,-0.794532,-0.627385,-0.227474,-0.025484,-0.025541,-0.025545,...,1.024638,-1.248813,-0.547609,-0.020565,0.000006,-0.020571,-0.021271,23.0,-2,59.0
1938857,2010-12-29,ZUMZ,ZUMIEZ INC,-0.025488,-0.874037,-1.248165,-0.238048,-0.025484,-0.025542,-0.025546,...,0.127223,-1.840589,0.494685,-0.013523,0.000006,-0.013529,-0.013398,67.0,-2,59.0
1938858,2010-12-30,ZUMZ,ZUMIEZ INC,-0.025489,-1.732662,-1.664124,-0.189909,-0.025484,-0.025543,-0.025547,...,-1.679840,-1.702722,-0.997203,0.000000,0.000006,-0.000006,0.000000,1541.0,2,59.0


In [28]:
data_20factors.to_csv('processed_data_20060101-20101231_20factors.csv')

In [29]:
data['ret_d'].mean()

0.0005448734716506539

In [30]:
data['ret_d'].std()

0.03289606561033

In [31]:
data['ret_d'].max()

1.849056603773585

In [32]:
data['ret_d'].min()

-0.9322916666666666